In [1]:
import functools
import numpy as np
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
tf.__version__

'2.3.1'

In [ ]:
train_file_path = "./trainset.csv"
test_file_path = "./testset.csv"

In [ ]:
LABEL_COLUMN = 'Application'
LABELS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [ ]:
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=20,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [ ]:
examples, labels = next(iter(raw_train_data))
# print("EXAMPLES: \n", examples, "\n")
# print("LABELS: \n", labels)

In [ ]:
def process_continuous_data(mean, data):
    data = tf.cast(data, tf.float32) * 1/(2 * mean)
    return tf.reshape(data, [-1, 1])

In [ ]:
MEANS = {
    'CPI_Rate': 1.2214,
    'Divider': 1,
    'Port_Utilization': 13.4,
    'Vector_Capacity_Usage': 1,
    'L1_Bound': 10.785,
    'L2_Bound': 1, 
    'L3_Bound': 1,
    'DRAM_Bound': 1,
    'Store_Bound': 2.94,
    'Front_End_Latency': 14.835, 
    'Front_End_Bandwidth': 11.37,
    'Light_Operations': 30.955,
    'Heavy_Operations': 1.66,
    'Bad_Speculations': 7.965, 
    'Average_CPU_Frequency': 3.765,
    'Total_Thread_Count': 98.25,
    'L1_Cache_Size': 0.256,
    'L2_Cache_Size': 1,
    'L3_Cache_Size': 8,
    'Number_of_Cores': 4,
    'DRAM_Bandwidth': 22,
    'DRAM_Capacity': 1
}

numerical_columns = []

for feature in MEANS.keys():
    num_col = tf.feature_column.numeric_column(
        feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
    numerical_columns.append(num_col)

In [ ]:
numerical_columns

In [ ]:
preprocessing_layer = tf.keras.layers.DenseFeatures(numerical_columns)

In [ ]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
train_data = raw_train_data
test_data = raw_test_data

In [ ]:
model.fit(train_data, epochs=30)

In [ ]:
model.summary()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

print()
print(f'Test Loss {test_loss}, Test Accuracy {test_accuracy}')

In [ ]:
predictions = model.predict(test_data)

predictions

In [ ]:
list(test_data)[0][1]